# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,qaqortoq,60.7167,-46.0333,6.76,87,100,2.94,GL,1690307298
1,1,axim,4.8699,-2.2405,25.34,88,97,3.63,GH,1690307118
2,2,iskushuban,10.2837,50.2300,32.20,33,17,14.67,SO,1690307490
3,3,santa ana,33.7456,-117.8678,37.82,70,20,2.06,US,1690307307
4,4,west island,-12.1568,96.8225,24.99,69,40,9.26,CC,1690307298


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [7]:
%%capture --no-display

# Configure the map plot
map_humidity =city_data_df.hvplot.points("Lng", "Lat", geo= True, tiles=True , size ="Humidity")

# Display the map
display(map_humidity)

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [30]:
# Narrow down cities that fit criteria and drop any results with null values
# Wind speed > 5, humidity > 90 , temp > 100
fantastic_weather = city_data_df[city_data_df['Humidity'] > 90]
fantastic_weather = fantastic_weather[fantastic_weather['Max Temp'] > 27]
fantastic_weather = fantastic_weather[fantastic_weather['Wind Speed'] > 6]
# Drop any rows with null values
fantastic_weather = fantastic_weather.dropna()


# Display sample data
display(fantastic_weather)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
60,60,saipan,15.1355,145.7010,28.39,100,75,6.26,MP,1690307334
97,97,ishigaki,24.3448,124.1572,28.07,94,75,9.26,JP,1690307505
379,379,san pedro,14.3500,121.0167,27.57,93,100,6.69,PH,1690307509
434,434,merizo village,13.2658,144.6691,28.31,91,100,8.90,GU,1690307099


### Step 3: Create a new DataFrame called `hotel_df`.

In [31]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = fantastic_weather.copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
display(hotel_df)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
60,60,saipan,15.1355,145.7010,28.39,100,75,6.26,MP,1690307334,
97,97,ishigaki,24.3448,124.1572,28.07,94,75,9.26,JP,1690307505,
379,379,san pedro,14.3500,121.0167,27.57,93,100,6.69,PH,1690307509,
434,434,merizo village,13.2658,144.6691,28.31,91,100,8.90,GU,1690307099,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [39]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey": geoapify_key,
    "categories": "accommodation.hotel"
    
    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{lng},{lat},{radius}'
    params["bias"] = f'proximity:{lng},{lat}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
saipan - nearest hotel: Chalan Kanoa Beach Hotel
ishigaki - nearest hotel: アートホテル石垣島
san pedro - nearest hotel: Hotel Takara
merizo village - nearest hotel: No hotel found


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
60,60,saipan,15.1355,145.7010,28.39,100,75,6.26,MP,1690307334,Chalan Kanoa Beach Hotel
97,97,ishigaki,24.3448,124.1572,28.07,94,75,9.26,JP,1690307505,アートホテル石垣島
379,379,san pedro,14.3500,121.0167,27.57,93,100,6.69,PH,1690307509,Hotel Takara
434,434,merizo village,13.2658,144.6691,28.31,91,100,8.90,GU,1690307099,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [44]:
%%capture --no-display

# Configure the map plot
map_hotel =hotel_df.hvplot.points("Lng", "Lat", geo= True, tiles=True , scale = 1, hover_cols =['Hotel Name', 'Country'] )

# Display the map
display(map_hotel)

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Hotel Name,Country)